In [158]:
import random as rd
import numpy as np
import h5py
import klepto
import shelve
import pickle
import scipy.io as sio
from scipy.sparse import *

## Input Prediction Model

In [159]:
import shelve
import pickle
with shelve.open("./1m/1 - ss/P_test.pkl", protocol=pickle.HIGHEST_PROTOCOL) as d:
    P_test_tree1 = d['content']
with shelve.open("./1m/1 - ss/rated_matrix.pkl", protocol=pickle.HIGHEST_PROTOCOL) as d:
    rated_matrix_tree1 = d['content']
P_test_tree1 = P_test_tree1.tocsc().toarray()

In [160]:
import shelve
import pickle
with shelve.open("./1m/2 - ss/P_test.pkl", protocol=pickle.HIGHEST_PROTOCOL) as d:
    P_test_tree2 = d['content']
with shelve.open("./1m/2 - ss/rated_matrix.pkl", protocol=pickle.HIGHEST_PROTOCOL) as d:
    rated_matrix_tree2 = d['content']
P_test_tree2 = P_test_tree2.tocsc().toarray()

In [161]:
import shelve
import pickle
with shelve.open("./1m/3 - ss/P_test.pkl", protocol=pickle.HIGHEST_PROTOCOL) as d:
    P_test_tree3 = d['content']
with shelve.open("./1m/3 - ss/rated_matrix.pkl", protocol=pickle.HIGHEST_PROTOCOL) as d:
    rated_matrix_tree3 = d['content']
P_test_tree3 = P_test_tree3.tocsc().toarray()

In [162]:
prediction_model = {1: {'P_test': P_test_tree1, 'rated_matrix': rated_matrix_tree1},
                    2: {'P_test': P_test_tree2, 'rated_matrix': rated_matrix_tree2},
                    3: {'P_test': P_test_tree3, 'rated_matrix': rated_matrix_tree3}}

## Input Test item info

In [163]:
dataset = '1m'

In [164]:
feature=h5py.File(dataset + '/test_list.mat')  
test_list = feature['test_list'][:]
test_list = list(map(int, test_list.T[0]))
test_list = [i-1 for i in test_list] 
feature=h5py.File(dataset + '/train_list.mat')
train_list = feature['train_list'][:]
train_list = list(map(int, train_list.T[0]))
train_list = [i-1 for i in train_list]

In [165]:
rating_matrix = load_npz('sparse_matrix_ml-' + dataset + '_selected.npz').tocsc()
rating_matrix_train = rating_matrix[:, train_list]
rating_matrix_test  = rating_matrix[:, test_list]
print("file load DONE")
rating_matrix_test = rating_matrix_test.toarray()

file load DONE


## Aggregate Three Tree models

In [196]:
lst1 = []
lst2 = []
lst3 = []
for itemid in range(rating_matrix_test.shape[1]):
    model_ind = rd.randint(1, 3)
    if model_ind == 1:
        lst1.append(itemid)
    elif model_ind == 2:
        lst2.append(itemid)
    elif model_ind == 3:
        lst3.append(itemid)

P_test1 = prediction_model[1]['P_test'][:, lst1]
P_test2 = prediction_model[2]['P_test'][:, lst2]
P_test3 = prediction_model[3]['P_test'][:, lst3]
rated_user1 = prediction_model[1]['rated_matrix'][:, lst1]
rated_user2 = prediction_model[2]['rated_matrix'][:, lst2]
rated_user3 = prediction_model[3]['rated_matrix'][:, lst3]
rating_matrix_test_unqueried1 = rating_matrix_test[:, lst1] * rated_user1
rating_matrix_test_unqueried2 = rating_matrix_test[:, lst2] * rated_user2
rating_matrix_test_unqueried3 = rating_matrix_test[:, lst3] * rated_user3

dif1 = P_test1*(rating_matrix_test_unqueried1!=0) - rating_matrix_test_unqueried1
dif2 = P_test2*(rating_matrix_test_unqueried2!=0) - rating_matrix_test_unqueried2
dif3 = P_test3*(rating_matrix_test_unqueried3!=0) - rating_matrix_test_unqueried3
RMSE1 = (dif1**2).sum()
RMSE2 = (dif2**2).sum()
RMSE3 = (dif3**2).sum()
RMSE_denominator1 = (rating_matrix_test_unqueried1!=0).sum()
RMSE_denominator2 = (rating_matrix_test_unqueried2!=0).sum()
RMSE_denominator3 = (rating_matrix_test_unqueried3!=0).sum() 
nominator1 = np.multiply(P_test1>3, rating_matrix_test_unqueried1>3).sum()
denominator1 = (rating_matrix_test_unqueried1>3).sum()
nominator2 = np.multiply(P_test2>3, rating_matrix_test_unqueried2>3).sum()
denominator2 = (rating_matrix_test_unqueried2>3).sum()
nominator3 = np.multiply(P_test3>3, rating_matrix_test_unqueried3>3).sum()
denominator3 = (rating_matrix_test_unqueried3>3).sum()

RMSE = ( (RMSE1 + RMSE2 + RMSE3) / (RMSE_denominator1 + RMSE_denominator2 + RMSE_denominator3) )**0.5
Precision = (nominator1 + nominator2 + nominator3) / (denominator1 + denominator2 + denominator3)

In [197]:
Precision

0.92295567526371403

In [198]:
RMSE

0.92662377613108093

## Aggregate Three Tree models (delete)

In [61]:
RMSE = 0
nominator = 0
denominator = 0
size = prediction_model[model_ind]['rated_matrix'].shape[0]
for itemid in range(rating_matrix_test.shape[1]):
    if itemid % 100 == 0:
        print(itemid)
    model_ind = rd.randint(1, 3)
    P_test = prediction_model[model_ind]['P_test'][:, itemid]
    rated_user = prediction_model[model_ind]['rated_matrix'][:, itemid].reshape(size, 1)
    rating_matrix_test_unqueried = rating_matrix_test[:, itemid] * rated_user
    dif = P_test*(rating_matrix_test_unqueried!=0) - rating_matrix_test_unqueried
    RMSE += ( (dif**2).sum() / (rating_matrix_test_unqueried!=0).sum() )**0.5
    nominator += np.multiply(P_test>3, rating_matrix_test_unqueried>3).sum()
    denominator += (rating_matrix_test_unqueried>3).sum()

0
100
200
300
400
500
600
700
800
900
1000
1100


In [62]:
Precision = nominator/denominator

In [63]:
Precision

0.92584767185488959